1 - 
Importer pandas as pd

In [65]:
import pandas as pd

### 📥 Fonction : `load_and_clean_csv(file_path)`

Cette fonction permet de **charger et nettoyer un fichier CSV** contenant des données sur les véhicules, notamment des informations liées à la consommation de carburant, aux émissions de CO2, à la puissance et à d'autres caractéristiques techniques.

#### 🔧 Étapes de traitement :
1. **Chargement des données**
   - Lit le fichier CSV en utilisant `;` comme séparateur et l'encodage `latin1`.

2. **Nettoyage des colonnes**
   - Standardise les noms de colonnes en les mettant en minuscules et en remplaçant les espaces par des underscores `_`.

3. **Conversion des champs numériques**
   - Convertit des colonnes clés comme `co2`, `conso_mixte`, `puiss_admin_98`, etc. en `float`, après remplacement des virgules par des points pour garantir la compatibilité.

4. **Conversion des dates**
   - Si elle existe, la colonne `date_maj` est convertie en format datetime avec gestion des erreurs.

5. **Suppression des doublons**
   - Élimine les lignes dupliquées dans le jeu de données.

#### 📤 Sortie :
- Retourne un **DataFrame Pandas** propre, prêt à être utilisé pour des analyses statistiques, visualisations ou entraînement de modèles de machine learning.

In [66]:
def load_and_clean_csv(file_path: str) -> pd.DataFrame:
    """
    Load and clean the vehicle CO2 dataset CSV file.

    Parameters:
    - file_path (str): Path to the CSV file.

    Returns:
    - pd.DataFrame: Cleaned and ready-to-use DataFrame.
    """
    # Load with appropriate encoding and separator
    df = pd.read_csv(file_path, sep=';', encoding='latin1')

    # Strip and standardize column names
    df.columns = [col.strip().lower().replace(" ", "_") for col in df.columns]

    # Convert numeric fields (e.g. consommation, CO2, puissances) to float
    numeric_cols = [
        'puiss_admin_98', 'puiss_max', 'conso_urb', 'conso_exurb', 'conso_mixte', 'co2',
        'hc', 'nox', 'hcnox', 'ptcl', 'masse_ordma_min', 'masse_ordma_max'
    ]
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', '.'), errors='coerce')

    # Optionally parse 'date_maj' if it’s a proper date column
    if 'date_maj' in df.columns:
        df['date_maj'] = pd.to_datetime(df['date_maj'], errors='coerce')

    # Drop duplicates if necessary
    df.drop_duplicates(inplace=True)

    # Return cleaned DataFrame
    return df
    

### 📥 Fonction : `load_and_clean_xls(file_path, sheet_name=0)`

Cette fonction permet de **charger et nettoyer un fichier Excel (XLS/XLSX)** contenant des données techniques et environnementales sur les véhicules.

Elle est utile dans le cadre d’un projet de data science ou machine learning traitant des émissions de CO2, de la consommation ou des caractéristiques techniques automobiles.

#### 🔧 Étapes de traitement :
1. **Chargement des données**
   - Lit le fichier Excel via `pandas.read_excel`, en prenant la première feuille par défaut (`sheet_name=0`), ou une feuille spécifique si un nom ou un index est fourni.

2. **Nettoyage des noms de colonnes**
   - Supprime les espaces inutiles
   - Met les noms en minuscules
   - Remplace les espaces par des underscores `_` pour faciliter la manipulation.

3. **Conversion des colonnes numériques**
   - Identifie et convertit des colonnes importantes (`co2`, `puiss_admin_98`, `conso_mixte`, etc.) en `float`, en remplaçant les virgules par des points.
   - Gère les erreurs de conversion (valeurs invalides deviennent `NaN`).

4. **Conversion de la date**
   - Si la colonne `date_maj` est présente, elle est convertie en format `datetime` avec gestion automatique des erreurs.

5. **Suppression des doublons**
   - Utilise `drop_duplicates()` pour supprimer les enregistrements identiques.

#### 📤 Sortie :
- Retourne un **DataFrame Pandas** propre et cohérent, prêt pour l’analyse, les visualisations ou l'entraînement de modèles prédictifs.


In [67]:
import locale
locale.setlocale(locale.LC_TIME, 'fr_FR.UTF-8')

def load_and_clean_xls(file_path: str, sheet_name: int | str = 0) -> pd.DataFrame:
    """
    Load and clean the vehicle CO2 dataset from an XLS file.

    Parameters:
    - file_path (str): Path to the XLS file.
    - sheet_name (int | str): Sheet index or name (default is 0 - first sheet).

    Returns:
    - pd.DataFrame: Cleaned and ready-to-use DataFrame.
    """
    # Load Excel sheet
    df = pd.read_excel(file_path, sheet_name=sheet_name)

    # Strip and standardize column names
    df.columns = [str(col).strip().lower().replace(" ", "_") for col in df.columns]

    # Convert numeric fields to float (if they exist)
    numeric_cols = [
        'puiss_admin_98', 'puiss_max', 'conso_urb', 'conso_exurb', 'conso_mixte', 'co2',
        'hc', 'nox', 'hcnox', 'ptcl', 'masse_ordma_min', 'masse_ordma_max'
    ]
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', '.'), errors='coerce')

    # Parse dates if applicable
    if 'date_maj' in df.columns:
        df['date_maj'] = pd.to_datetime(df['date_maj'], format='%B %Y', errors='coerce')

    # Drop duplicates
    df.drop_duplicates(inplace=True)

    return df

### ⚙️ Configuration du chemin PYTHONPATH

Ce bloc de code est utilisé pour **ajouter dynamiquement le répertoire racine du projet au PYTHONPATH**, afin de permettre l'import de modules personnalisés situés en dehors du répertoire courant.

#### 🧱 Détail du fonctionnement :
1. **Importation des modules système**
   - `sys` : pour manipuler les chemins d’accès utilisés par l'interpréteur Python.
   - `os` et `pathlib.Path` : pour manipuler les chemins de fichiers et répertoires de manière robuste et multiplateforme.

2. **Détection du répertoire racine du projet**
   - `Path(__file__).parent.parent` : part du fichier en cours d’exécution (`__file__`), remonte de deux niveaux pour atteindre le dossier racine du projet (ex : `mon_projet/`).

3. **Ajout au PYTHONPATH**
   - `sys.path.append(...)` : ajoute le chemin obtenu à la liste `sys.path` pour que Python puisse détecter les modules personnalisés même s’ils ne sont pas dans le dossier d'exécution.


## Les données de 2001

In [68]:
df_edition_2001 = load_and_clean_xls('fic_etiq_edition_2001.xls')
df_edition_2001.head()

,marques,modele,typ._mines,cnit,carb,cv,puiss.,bv,urbain,ex.urb,mixte,co2
0,VOLKSWAGEN,LUPO 3L TDI,MVW70C1R4385,6ESCANYX01AGFD5850021N0H,GO,3,45.0,A 5,3.6,2.7,3.0,81
1,VOLKSWAGEN,LUPO 3L TDI,MVW70C1RX572,6ESCAYZX01AGFD5850021N0H,GO,3,45.0,A 5,3.6,2.7,3.0,81
2,VOLKSWAGEN,LUPO 3L TDI,MVW70C1R5386,6ESCANYX01AGFD5850021N0I,GO,3,45.0,A 5,3.8,2.8,3.2,86
3,VOLKSWAGEN,LUPO 3L TDI,MVW70C1R6387,6ESCANYX01AGFD5850021N1H,GO,3,45.0,A 5,4.0,2.7,3.2,86
4,VOLKSWAGEN,LUPO 3L TDI,MVW70C1RY573,6ESCAYZX01AGFD5850021N0I,GO,3,45.0,A 5,3.8,2.8,3.2,86


## Les données de 2002

In [69]:
df_edition_2002 = load_and_clean_xls('fic_etiq_edition_2002.xls')
df_edition_2002.head()

,groupe,marque,modele_version,cnit,unnamed:_4,unnamed:_5,mine,energie,puissance_fiscale,puissance_reelle,bv,urb,ex-urb,mixte,co2,immat,segment
0,NaN,ALFA-ROMEO,147 3P 1.6TS (105ch),MAL1301JL265,1301JL265,1301J265,937AXA1A00,ES,7,77.0,M 5,11.1,6.3,8.1,192.0,NaN,NaN
1,NaN,ALFA-ROMEO,147 3P 1.6TS (105ch),MAL1301JP330,1301JP330,1301J330,937AXA1A00B,ES,7,77.0,M 5,11.3,6.4,8.2,196.0,NaN,NaN
2,NaN,ALFA-ROMEO,147 5P 1.6TS (105ch),MAL1302JY276,1302JY276,1302J276,937BXA1A05,ES,7,77.0,M 5,11.1,6.3,8.1,192.0,NaN,NaN
3,NaN,ALFA-ROMEO,147 5P 1.6TS (105ch),MAL1302JU333,1302JU333,1302J333,937BXA1A05B,ES,7,77.0,M 5,11.3,6.4,8.2,196.0,NaN,NaN
4,NaN,ALFA-ROMEO,147 3P 1.6TS (120ch),MAL1401JW266,1401JW266,1401J266,937AXB1A01,ES,8,88.0,M 5,11.2,6.4,8.2,194.0,NaN,NaN


## Les données de 2003

In [70]:
df_edition_2003 = load_and_clean_xls('fic_etiq_edition_2003.xls')
df_edition_2003.head()

,marque,modele_version,cnit,mine,energie,puissance_fiscale,puissance_reelle,bv,urb,ex-urb,mixte,co2,immat
0,ALFA-ROMEO,147 3P 1.6TS (105ch),MAL1301JL265,937AXA1A00,ES,7,77.0,M 5,11.1,6.3,8.1,192.0,2003-03-01
1,ALFA-ROMEO,147 3P 1.6TS (105ch),MAL1301JE355,937AXA1A00C,ES,7,77.0,M 5,11.1,6.3,8.1,192.0,2003-03-01
2,ALFA-ROMEO,147 3P 1.6TS (120ch),MAL1401JW266,937AXB1A01,ES,8,88.0,M 5,11.2,6.4,8.2,194.0,2003-03-01
3,ALFA-ROMEO,147 3P 1.6TS (120ch),MAL1401JQ357,937AXB1A01C,ES,8,88.0,M 5,11.2,6.4,8.2,194.0,2003-03-01
4,ALFA-ROMEO,147 3P 1.9JTD,MAL5401JG274,937AXD1A04,GO,7,85.0,M 5,7.8,4.7,5.8,155.0,2003-03-01


## Les données de 2004

In [71]:
df_edition_2004 = load_and_clean_xls('fic_etiq_edition_2004.xls')
df_edition_2004.head()

,marques,"modèles,_versions",cnit,carburant,puiss_administrative,puissance_max,bv,conso_urb,conso_ex-urb,conso_mixte,co2,unnamed:_11
0,VOLKSWAGEN,LUPO 3L TDI (61ch) TIPTRONIC,MVW70K1RU464,GO,3,45.0,A 5,3.6,2.7,3.0,81,Décembre 2003
1,VOLKSWAGEN,LUPO 3L TDI (61ch) TIPTRONIC,MVW70K1RX467,GO,3,45.0,A 5,4.2,2.8,3.3,88,Décembre 2003
2,SMART,SMART CDI,MST5001A5074,GO,3,30.0,M 6,3.9,3.1,3.4,90,Décembre 2003
3,SMART,SMART CDI CABRIOLET,MST5005AL082,GO,3,30.0,M 6,3.9,3.1,3.4,90,Décembre 2003
4,SMART,SMART CDI,MST5001A6075,GO,3,30.0,M 6,4.0,3.3,3.6,95,Décembre 2003


## Les données de 2005

In [83]:
df_edition_2005 = load_and_clean_xls('fic_etiq_edition_2005.xls')
df_edition_2005.head()

,marque,modele_version,cnit,puissance_fiscale,puissance_reelle,bv,urb,ex-urb,mixte,co2,energie
0,ALFA-ROMEO,147 3P 1.6 TS (105ch),MAL1301JY594,7,77.0,M 5,11.1,6.3,8.1,192,ES
1,ALFA-ROMEO,147 3P 1.6 TS (120ch),MAL1401JA596,8,88.0,M 5,11.2,6.4,8.2,194,ES
2,ALFA-ROMEO,147 3P 2.0 TS,MAL1601JY600,10,110.0,M 5,12.1,7.0,8.9,211,ES
3,ALFA-ROMEO,147 3P 2.0L SELESPEED,MAL1601JW598,10,110.0,M 5,12.1,7.0,8.9,211,ES
4,ALFA-ROMEO,147 5P 1.6 TS (105ch),MAL1302JC606,7,77.0,M 5,11.1,6.3,8.1,192,ES


## Les données de 2006

In [73]:
df_edition_2006 = load_and_clean_xls('fic_etiq_edition_2006.xls')
df_edition_2006.head()

,alfa-romeo,147_3p_1.6l_ts_(105ch),mal1301jz595,937axa1a00h,es,7,77.0,m_5,11.3,6.4,8.2,196,janvier_2006
0,ALFA-ROMEO,147 3P 1.6L TS (120ch),MAL1401JB597,937AXB1A01H,ES,8,88.0,M 5,11.2,6.4,8.2,194.0,Janvier 2006
1,ALFA-ROMEO,147 3P 2.0L SELESPEED,MAL1601JX599,937AXC1102H,ES,10,110.0,M 5,12.1,7.0,8.9,211.0,Janvier 2006
2,ALFA-ROMEO,147 3P 2.0L TS,MAL1601JZ601,937AXC1A03H,ES,10,110.0,M 5,12.1,7.0,8.9,211.0,Janvier 2006
3,ALFA-ROMEO,147 3P JTD (120ch) M-JET,MAL5401JH627,937AXU1A32,GO,7,88.0,M 5,7.8,4.7,5.8,153.0,Janvier 2006
4,ALFA-ROMEO,147 3P JTD (150ch) M-JET,MAL5601J0626,937AXN1B30B,GO,9,110.0,M 6,8.0,4.7,5.9,157.0,Janvier 2006


## Les données de 2007

In [74]:
df_edition_2007 = load_and_clean_xls('fic_etiq_edition_2007.xls')
df_edition_2007.head()

,marque,modele_version,cnit,puissance_fiscale,puissance_reelle,bv,urb,ex-urb,mixte,co2,carburant,etiquette_energie
0,ALFA-ROMEO,147 3P 1.6L TS (105ch),MAL1301JZ595,7,77.0,M 5,11.3,6.4,8.2,196,ES,E
1,ALFA-ROMEO,147 3P 1.6L TS (120ch),MAL1401JB597,8,88.0,M 5,11.2,6.4,8.2,194,ES,E
2,ALFA-ROMEO,147 3P 2.0L SELESPEED,MAL1601JX599,10,110.0,M 5,12.1,7.0,8.9,211,ES,F
3,ALFA-ROMEO,147 3P 2.0L TS,MAL1601JZ601,10,110.0,M 5,12.1,7.0,8.9,211,ES,F
4,ALFA-ROMEO,147 3P JTD (120ch) M-JET,MAL5401JH627,7,88.0,M 5,7.8,4.7,5.8,153,GO,D


## Les données de 2008

In [75]:
df_edition_2008 = load_and_clean_xls('fic_etiq_edition_2008.xls')
df_edition_2008.head()

,marque,modele_version,cnit,puissance_fiscale,puissance_reelle,bv,urb,ex-urb,mixte,co2,carburant,etiquette_energie,bonus(-)/malus(_),type_2
0,DAIHATSU,CUORE II BVM5,MJD1102JU185,4.0,51.0,M 5,5.5,3.8,4.4,104.0,ES,B,-700,MJD1102J
1,TOYOTA,PRIUS (75ch) VVT-i,MJT9162TM872,4.0,57.0,V 0,5.0,4.2,4.3,104.0,ES,B,-2000,MJT9162T
2,TOYOTA,PRIUS (75ch) VVT-i,MJT9172TT515,4.0,57.0,V 0,5.0,4.2,4.3,104.0,ES,B,-2000,MJT9172T
3,CITROEN,C1 3P 1.0i,MCT1011TN299,4.0,50.0,M 5,5.5,4.1,4.6,108.0,ES,B,-700,MCT1011T
4,CITROEN,C1 3P 1.0i SensoDrive,MCT3011TU300,4.0,50.0,A 5,5.5,4.1,4.6,108.0,ES,B,-700,MCT3011T


## Les données de 2009

In [84]:
df_edition_2009 = load_and_clean_xls('fic_etiq_edition_2009.xls')
df_edition_2009.head()

,marque,modele_version,cnit,puissance_fiscale,puissance_reelle,bv,urb,ex-urb,mixte,co2,carburant,etiquette_energie,bonus(-)/malus(+)
0,ALFA-ROMEO,MiTo 1.4 MPI (78ch),MAL1101P0804,5,58.0,M 6,7.7,4.8,5.9,138,ES,C,-
1,ALFA-ROMEO,MiTo 1.4 MPI (78ch),MAL1101PZ803,5,58.0,M 6,7.7,4.8,5.9,138,ES,C,-
2,ALFA-ROMEO,147 3P 1.6L TS (105ch),MAL1301JZ595,7,77.0,M 5,11.3,6.4,8.2,196,ES,E,+ 750
3,ALFA-ROMEO,147 5P 1.6TS (105ch),MAL1302JD607,7,77.0,M 5,11.3,6.4,8.2,196,ES,E,+ 750
4,ALFA-ROMEO,147 3P 1.6L TS (120ch),MAL1401JB597,8,88.0,M 5,11.2,6.4,8.2,194,ES,E,+ 750


## Les données de 2010

In [85]:
df_edition_2010 = load_and_clean_xls('fic_etiq_edition_2010.xls')
df_edition_2010.head()

,marque,modele_version,cnit,puissance_fiscale,puissance_reelle,bv,urb,ex-urb,mixte,co2,carburant,etiquette_energie,bonus(-)/malus(+)
0,SMART,SMART FORTWO COUPE ELECTRIC,M10SMTVP000E039,5,NaN,M 1,NaN,NaN,NaN,NaN,EL,A,-5000
1,SMART,SMART FORTWO Cabrio ELECTRIC,M10SMTVP000F040,5,NaN,M 1,NaN,NaN,NaN,NaN,EL,A,-5000
2,ALFA-ROMEO,MiTo 1.4 MPI (105ch) MultiAir,M10ALFVP0002035,6,77.0,M 6,7.6,4.8,5.8,136.0,ES,C,-
3,ALFA-ROMEO,GT COUPE 2.0L JTS,M10ALFVP0002132,11,122.0,M 5,12.2,6.7,8.7,207.0,ES,F,+ 1600
4,ALFA-ROMEO,GT COUPE 2.0L JTS,M10ALFVP0003133,11,122.0,M 5,12.2,6.7,8.7,207.0,ES,F,+ 1600


## Les données de 2011

In [78]:
df_edition_2011 = load_and_clean_xls('fic_etiq_edition_2011.xls')
df_edition_2011.head()

,lib_mrq,lib_mod_doss,lib_mod,dscom,cnit,tvv,typ_cbr,puiss_admin_98,puiss_max,typ_boite_nb_rapp,conso_urb,conso_exurb,conso_mixte,co2,champ_v9,date_maj
0,ALFA-ROMEO,147,ALFA 147,147 1.6 TS (105ch),M10ALFVP000M091,937BXA1A05L,ES,7,77.0,M 5,11.3,6.4,8.2,196.0,70/220*2003/76EURO4,2011-03-01
1,ALFA-ROMEO,147,ALFA 147,147 1.6 TS (120ch),M10ALFVP000P093,937BXB1A06L,ES,8,88.0,M 5,11.2,6.4,8.2,194.0,70/220*2003/76EURO4,2011-03-01
2,ALFA-ROMEO,147,ALFA 147,147 3P 1.6 TS (105ch),MAL1301JZ595,937AXA1A00H,ES,7,77.0,M 5,11.3,6.4,8.2,196.0,70/220*2003/76EURO4,2011-03-01
3,ALFA-ROMEO,147,ALFA 147,147 3P 1.6 TS (120ch),MAL1401JB597,937AXB1A01H,ES,8,88.0,M 5,11.2,6.4,8.2,194.0,70/220*2003/76EURO4,2011-03-01
4,ALFA-ROMEO,147,ALFA 147,147 3P 2.0 SELESPEED,MAL1601JX599,937AXC1102H,ES,10,110.0,M 5,12.1,7.0,8.9,211.0,70/220*2002/80EURO4,2011-03-01


## Les données de 2012

In [79]:
df_clean_2012 = load_and_clean_csv('fic_etiq_edition_2012.csv')
df_clean_2012.head()

,lib_mrq,lib_mod_doss,lib_mod,dscom,cnit,tvv,typ_cbr,hybride,puiss_admin_98,puiss_max,...,hc,nox,hcnox,ptcl,masse_ordma_min,masse_ordma_max,champ_v9,date_maj,carrosserie,gamme
0,ALFA-ROMEO,GIULIETTA,GIULIETTA,GIULIETTA 1.4 T-jet (120ch) S/S,M10ALFVP0000324,940FXF1A05,ES,non,7,85,...,0.030,0.026,NaN,NaN,1355,1355,715/2007*692/2008EURO5,NaT,BERLINE,MOY-SUPER
1,ALFA-ROMEO,MITO,MITO,MITO 0.9 Twin Air (85ch) S/S,M10ALFVP0000360,955AXW1B17B,ES,non,4,62,...,0.049,0.037,NaN,NaN,1205,1205,715/2007*692/2008EURO5,NaT,BERLINE,INFERIEURE
2,ALFA-ROMEO,GIULIETTA,GIULIETTA,GIULIETTA 1.4 T-jet (120ch) S/S,M10ALFVP0001325,940FXF1A05B,ES,non,7,85,...,0.030,0.026,NaN,NaN,1355,1355,715/2007*692/2008EURO5,NaT,BERLINE,MOY-SUPER
3,ALFA-ROMEO,MITO,MITO,MITO 1.4 MPI (105ch) MultiAir,M10ALFVP0002035,955AXL1B08,ES,non,6,77,...,0.049,0.040,NaN,NaN,1185,1185,715/2007*692/2008EURO5,NaT,BERLINE,INFERIEURE
4,ALFA-ROMEO,GIULIETTA,GIULIETTA,GIULIETTA 1.4 TB MultiAir (170ch) S/S,M10ALFVP0002326,940FXG1A06,ES,non,9,120,...,0.048,0.029,NaN,NaN,1365,1365,715/2007*692/2008EURO5,NaT,BERLINE,MOY-SUPER


## Les données de 2013

In [80]:
df_clean_2013 = load_and_clean_csv('fic_etiq_edition_2013.csv')
df_clean_2013.head()

,marque,modèle_dossier,modèle_utac,désignation_commerciale,cnit,type_variante_version_(tvv),carburant,hybride,puissance_administrative,puissance_maximale_(kw),...,hc_(g/km),nox_(g/km),hc+nox_(g/km),particules_(g/km),masse_vide_euro_min_(kg),masse_vide_euro_max_(kg),champ_v9,date_de_mise_à_jour,carrosserie,gamme
0,ALFA-ROMEO,159,159,159 1750 Tbi (200ch),M10ALFVP000G340,939AXN1B52C,ES,non,12,147.0,...,0.052,0.032,NaN,0.002,1505,1505,715/2007*692/2008EURO5,juin-13,BREAK,MOY-SUPER
1,ALFA-ROMEO,159,159,159 2.0 JTDm (170ch) ECO,M10ALFVP000U221,939AXP1B54C,GO,non,9,125.0,...,NaN,0.169,0.190,0.003,1565,1565,715/2007*692/2008EURO5,juin-13,BERLINE,MOY-SUPER
2,ALFA-ROMEO,159,159,159 2.0 JTDm (136ch),M10ALFVP000E302,939AXR1B64,GO,non,7,100.0,...,NaN,0.149,0.175,0.001,1565,1565,715/2007*692/2008EURO5,juin-13,BERLINE,MOY-SUPER
3,ALFA-ROMEO,159,159,159 2.0 JTDm (136ch),M10ALFVP000F303,939AXR1B64B,GO,non,7,100.0,...,NaN,0.149,0.175,0.001,1565,1565,715/2007*692/2008EURO5,juin-13,BERLINE,MOY-SUPER
4,ALFA-ROMEO,159,159,159 2.0 JTDm (170ch),M10ALFVP000G304,939AXS1B66,GO,non,9,125.0,...,NaN,0.164,0.193,0.001,1565,1565,715/2007*692/2008EURO5,juin-13,BERLINE,MOY-SUPER


## Les données de 2014

In [81]:
df_clean_2014 = load_and_clean_csv('fic_etiq_edition_2014.csv')
df_clean_2014.head()

/var/folders/l2/99pj_2g16y5_0njpmww3yp280000gn/T/ipykernel_40958/4172475450.py:28: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date_maj'] = pd.to_datetime(df['date_maj'], errors='coerce')


,lib_mrq,lib_mod_doss,lib_mod,dscom,cnit,tvv,cod_cbr,hybride,puiss_admin_98,puiss_max,...,masse_ordma_min,masse_ordma_max,champ_v9,date_maj,carrosserie,gamme,unnamed:_26,unnamed:_27,unnamed:_28,unnamed:_29
0,ALFA-ROMEO,159,159,159 1750 Tbi (200ch),M10ALFVP000G340,939AXN1B52C,ES,non,12,147.0,...,1505,1505,715/2007*692/2008EURO5,NaT,BERLINE,MOY-SUPER,NaN,NaN,NaN,NaN
1,ALFA-ROMEO,159,159,159 1750 Tbi (200ch),M10ALFVP000H341,939BXN1B53C,ES,non,12,147.0,...,1555,1555,715/2007*692/2008EURO5,NaT,BERLINE,MOY-SUPER,NaN,NaN,NaN,NaN
2,ALFA-ROMEO,159,159,159 2.0 JTDm (136ch),M10ALFVP000E302,939AXR1B64,GO,non,7,100.0,...,1565,1565,715/2007*692/2008EURO5,NaT,BERLINE,MOY-SUPER,NaN,NaN,NaN,NaN
3,ALFA-ROMEO,159,159,159 2.0 JTDm (136ch),M10ALFVP000F303,939AXR1B64B,GO,non,7,100.0,...,1565,1565,715/2007*692/2008EURO5,NaT,BERLINE,MOY-SUPER,NaN,NaN,NaN,NaN
4,ALFA-ROMEO,159,159,159 2.0 JTDm (170ch),M10ALFVP000G304,939AXS1B66,GO,non,9,125.0,...,1565,1565,715/2007*692/2008EURO5,NaT,BERLINE,MOY-SUPER,NaN,NaN,NaN,NaN


## Les données de 2015

In [82]:
df_clean_2015 = load_and_clean_csv('fic_etiq_edition_2015.csv')
df_clean_2015.head()

/var/folders/l2/99pj_2g16y5_0njpmww3yp280000gn/T/ipykernel_40958/4172475450.py:28: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date_maj'] = pd.to_datetime(df['date_maj'], errors='coerce')


,lib_mrq_doss,lib_mod_doss,mrq_utac,mod_utac,dscom,cnit,tvv,energ,hybride,puiss_admin,...,co2_mixte,co_typ_1,hc,nox,hcnox,ptcl,masse_ordma_min,masse_ordma_max,champ_v9,date_maj
0,ALFA ROMEO,159,ALFA ROMEO,159,159 1750 Tbi (200ch),M10ALFVP000G340,939AXN1B52C,ES,non,12,...,182.0,0.647,0.052,0.032,NaN,0.002,1505,1505,715/2007*692/2008EURO5,NaT
1,ALFA ROMEO,159,ALFA ROMEO,159,159 2.0 JTDm (170ch) ECO,M10ALFVP000U221,939AXP1B54C,GO,non,9,...,136.0,0.192,NaN,0.169,0.190,0.003,1565,1565,715/2007*692/2008EURO5,NaT
2,ALFA ROMEO,159,ALFA ROMEO,159,159 2.0 JTDm (136ch),M10ALFVP000E302,939AXR1B64,GO,non,7,...,134.0,0.066,NaN,0.149,0.175,0.001,1565,1565,715/2007*692/2008EURO5,NaT
3,ALFA ROMEO,159,ALFA ROMEO,159,159 2.0 JTDm (136ch),M10ALFVP000F303,939AXR1B64B,GO,non,7,...,134.0,0.066,NaN,0.149,0.175,0.001,1565,1565,715/2007*692/2008EURO5,NaT
4,ALFA ROMEO,159,ALFA ROMEO,159,159 2.0 JTDm (170ch),M10ALFVP000G304,939AXS1B66,GO,non,9,...,139.0,0.060,NaN,0.164,0.193,0.001,1565,1565,715/2007*692/2008EURO5,NaT
